# Jupyter Notebook - CDM Data Aggregation Toolkit

**Purpose**: This notebook provides functions for generating summaries for all datasets in CDM

With the exception of data preparation, this notebook can be used as is. Users will be required to prepare data in the required structure and specify paths.

**Edit Date** : 04/27/2020

Please refer to following example: https://github.pwc.com/GlobalAnalyticApps/Verizon_Project/blob/Phase_II_Mathew/Needs%20Profiling_Sub%20Profiles_MG-Functions.ipynb

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
import argparse
import pandas as pd
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import col
from ipywidgets import widgets
from IPython.display import display
#from IPython.html.widgets import interactive

from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import clear_output

import warnings
warnings.filterwarnings("ignore")

from IPython.core.display import display, HTML
from IPython.display import clear_output

In [ ]:
spark = SparkSession.builder\
                   .appName("CDM Data Aggregation Toolkit - Digital Lab")\
                   .config("spark.dynamicAllocation.enabled", "true")\
                   .config("spark.shuffle.service.enabled", "true")\
                   .config("spark.executor.cores","5")\
                   .config("spark.sql.crossJoin.enabled","true")\
                   .config("spark.executor.memory", "12G")\
                   .config("spark.driver.memory", "8G")\
                   .config("spark.sql.autobroadcastjointhreshold","33554432")\
                   .config("spark.speculation","true")\
                   .enableHiveSupport()\
                   .getOrCreate()

## 1. Functions

#### 1.1 Wide to Long Form conversion

In [ ]:
#function to melt a dataframe
def melt(df, id_vars, value_vars, var_name, value_name):
    """Convert :class:`DataFrame` from wide to long format."""

    # Create array<struct<variable: str, value: ...>>
    _vars_and_vals = array(*(struct(lit(c).alias(var_name), col(c).alias(value_name)) 
       for c in value_vars))

    # Add to the DataFrame and explode
    _tmp = df.withColumn("_vars_and_vals", explode(_vars_and_vals))

    cols = id_vars + [ col("_vars_and_vals")[x].alias(x) for x in [var_name, value_name]]
    return _tmp.select(*cols)

#### 1.2 Claritas Data preperation

In [ ]:
def cla_summary_master(claritas_df,df,distinct_id,need_id,col_list):
    cla_var_df = spark.createDataFrame(col_list, StringType())
    cla_var_df = cla_var_df.withColumnRenamed('value','variable_id')
    claritas_df = claritas_df.join(broadcast(cla_var_df),'variable_id')    
    claritas_vars_long = df.select(distinct_id,'segment_code',need_id).distinct()  
    claritas_vars_long = claritas_vars_long\
                                  .join(broadcast(claritas_df).filter(col('segment_code')!=99),'segment_code')
    return claritas_vars_long

#### 1.3 Claritas summary generation

In [ ]:
def cla_summary(claritas_df,df,distinct_id,need_id,col_list):
    claritas_vars_long = cla_summary_master(claritas_df,df,distinct_id,need_id,col_list)
    cla_cont_df = claritas_vars_long.groupBy(need_id,"variable_id")\
                                        .agg(*[sum(col(column)).alias(column) for column in ["numerator","denominator"]])\
                                        .orderBy(need_id,"variable_id")
    cla_cont_df = cla_cont_df\
                            .withColumn("value",col("numerator")/(col("denominator")*100)).drop('numerator','denominator')
    cla_cont_df = cla_cont_df.withColumn('bucket',lit("")).select('variable_id','bucket',need_id,'value')
    cla_cont_df = cla_cont_df.groupBy('variable_id','bucket').pivot(need_id).agg(avg('value'))
    cla_total_cont_df = claritas_vars_long.groupBy("variable_id")\
                                        .agg(*[sum(col(column)).alias(column) for column in ["numerator","denominator"]])\
                                        .orderBy("variable_id")
    cla_total_cont_df = cla_total_cont_df\
                        .withColumn("value",col("numerator")/(col("denominator")*100)).drop('numerator','denominator')
    cla_total_cont_df = cla_total_cont_df.groupBy('variable_id').agg(avg('value').alias('total'))
    cla_total_cont_df = cla_total_cont_df.withColumn('bucket',lit("")).select('variable_id','bucket','total')
    cla_df = cla_cont_df.join(cla_total_cont_df,['variable_id','bucket'])
    return cla_df

#### 1.4 Full summary generation

In [ ]:
def summary(claritas_df,df,need_id,ind_cont_list,ind_cat_list,hh_cont_list,hh_cat_list,ind_cla_list,hh_cla_list):
    pop = df.withColumn('variable_id',lit('ind_pop')).withColumn('bucket',lit(""))\
                .select('person_id',need_id,'variable_id','bucket')
    ind_pop = pop.groupBy('variable_id','bucket').pivot(need_id).agg(count('person_id').alias('pop'))
    total_ind_pop = pop.groupBy('variable_id','bucket').agg(count('person_id').alias('total'))
    ind_pop = ind_pop.join(total_ind_pop,['variable_id','bucket'],how='left')
    
    pop_2 = df.withColumn('variable_id',lit('hh_pop')).withColumn('bucket',lit(""))\
                .select('ig_household_id',need_id,'variable_id','bucket')
    hh_pop = pop_2.groupBy('variable_id','bucket').pivot(need_id).agg(countDistinct('ig_household_id').alias('pop'))
    total_hh_pop = pop_2.groupBy('variable_id','bucket').agg(countDistinct('ig_household_id').alias('total'))
    hh_pop = hh_pop.join(total_hh_pop,['variable_id','bucket'],how='left')
     
    summary_df = ind_pop.unionAll(hh_pop)
    
    if len(ind_cont_list)>0:
        ind_cont = df.select('person_id',need_id,\
                             *(col(c).cast('double') for c in ind_cont_list)).distinct().orderBy(need_id)
        ind_cont = melt(ind_cont,['person_id',need_id],ind_cont_list,var_name='variable_id',value_name='value')
        ind_cont = ind_cont.withColumn('bucket',lit("")).select('variable_id','bucket',need_id,'value')
        ind_cont = ind_cont.groupBy('variable_id','bucket').pivot(need_id).agg(avg('value').alias('avg'))\
                            .orderBy('variable_id')
        total_ind_cont = df.select('person_id',*(col(c).cast('double') for c in ind_cont_list)).distinct()
        total_ind_cont = melt(total_ind_cont,['person_id'],ind_cont_list,var_name='variable_id',value_name='value')
        total_ind_cont = total_ind_cont.groupBy('variable_id').agg(avg('value')\
                                                                   .alias('total')).orderBy('variable_id')
        total_ind_cont = total_ind_cont.withColumn('bucket',lit("")).select('variable_id','bucket','total')
        ind_cont = ind_cont.join(total_ind_cont,['variable_id','bucket'],how='left')
        summary_df = summary_df.unionAll(ind_cont)
    
    if len(ind_cat_list)>0:    
        ind_cat = df.select('person_id',need_id,*(col(c).cast('string') for c in ind_cat_list)).distinct().orderBy(need_id)
        ind_cat = melt(ind_cat,['person_id',need_id],ind_cat_list,var_name='variable_id',value_name='bucket')
        ind_cat = ind_cat.groupBy('variable_id','bucket').pivot(need_id)\
                   .agg(count('person_id').alias('count')).orderBy('variable_id','bucket')
        total_ind_cat = df.select('person_id',*(col(c).cast('string') for c in ind_cat_list)).distinct()
        total_ind_cat = melt(total_ind_cat,['person_id'],ind_cat_list,var_name='variable_id',value_name='bucket')
        total_ind_cat = total_ind_cat.groupBy('variable_id','bucket').agg(count('person_id')\
                                                                  .alias('total')).orderBy('variable_id','bucket')
        ind_cat = ind_cat.join(total_ind_cat,['variable_id','bucket'],how='left')
        summary_df = summary_df.unionAll(ind_cat)
    
    if len(hh_cont_list)>0:
        hh_cont = df.select('ig_household_id',need_id,\
                             *(col(c).cast('double') for c in hh_cont_list)).distinct().orderBy(need_id)
        hh_cont = melt(hh_cont,['ig_household_id',need_id],hh_cont_list,var_name='variable_id',value_name='value')
        hh_cont = hh_cont.withColumn('bucket',lit("")).select('variable_id','bucket',need_id,'value')
        hh_cont = hh_cont.groupBy('variable_id','bucket').pivot(need_id).agg(avg('value').alias('avg'))\
                            .orderBy('variable_id')
        total_hh_cont = df.select('ig_household_id',*(col(c).cast('double') for c in hh_cont_list)).distinct()
        total_hh_cont = melt(total_hh_cont,['ig_household_id'],hh_cont_list,var_name='variable_id',value_name='value')
        total_hh_cont = total_hh_cont.groupBy('variable_id').agg(avg('value')\
                                                                   .alias('total')).orderBy('variable_id')
        total_hh_cont = total_hh_cont.withColumn('bucket',lit("")).select('variable_id','bucket','total')
        hh_cont = hh_cont.join(total_hh_cont,['variable_id','bucket'],how='left')
        summary_df = summary_df.unionAll(hh_cont)
    
    if len(hh_cat_list)>0:    
        hh_cat = df.select('ig_household_id',need_id,*(col(c).cast('string') for c in hh_cat_list))\
                    .distinct().orderBy(need_id)
        hh_cat = melt(hh_cat,['ig_household_id',need_id],hh_cat_list,var_name='variable_id',value_name='bucket')
        hh_cat = hh_cat.groupBy('variable_id','bucket').pivot(need_id)\
                   .agg(count('ig_household_id').alias('count')).orderBy('variable_id','bucket')
        total_hh_cat = df.select('ig_household_id',*(col(c).cast('string') for c in hh_cat_list)).distinct()
        total_hh_cat = melt(total_hh_cat,['ig_household_id'],hh_cat_list,var_name='variable_id',value_name='bucket')
        total_hh_cat = total_hh_cat.groupBy('variable_id','bucket').agg(count('ig_household_id')\
                                                                  .alias('total')).orderBy('variable_id','bucket')
        hh_cat = hh_cat.join(total_hh_cat,['variable_id','bucket'],how='left')
        summary_df = summary_df.unionAll(hh_cat)
    
    if len(ind_cla_list)>0:
        ind_cla_df = cla_summary(claritas_df,df,'person_id',need_id,ind_cla_list)
        summary_df = summary_df.unionAll(ind_cla_df)
    
    if len(hh_cla_list)>0:
        hh_cla_df = cla_summary(claritas_df,df,'ig_household_id',need_id,hh_cla_list)
        summary_df = summary_df.unionAll(hh_cla_df)
    
    return summary_df

## 2. Reading Data for summaries

#### 2.1 Reading Claritas Dataframe
Structure: (Long form)
* segment_code - claritas segment (psp_seg_id/psn_seg_id)
* cla_ndp_001_c - household population (available in CDM)
* adult_count - average adult count per segment
* variable_id - names of all variables to be aggregated
* value - values of all variables to be aggregated
* numerator - value * adult population/hh population
* denominator - adult population/hh population

##### standard sample claritas input file - please mention path here:

In [ ]:
claritas_path = '.csv'

In [ ]:
claritas_df = pd.read_csv(claritas_path)

In [ ]:
claritas_df = spark.createDataFrame(claritas_df)

#### 2.2 Reading Integrated Dataframe for all other Datasets in CDM
Structure: (Wide Form)
* person_id - unique individual identifier (available in CDM)
* ig_household_id - unique household identifier (available in CDM)
* segment_code - claritas segment (psp_seg_id/psn_seg_id)
* Data cut required - eg: Segment, State etc
* All variables to be aggregated

##### standard sample infogroup input file - please mention path here:

In [ ]:
df_path = '.csv'

In [ ]:
df = pd.read_csv(df_path)

In [ ]:
df = spark.createDataFrame(df)

#### 2.3 Initializing List of variables to be aggregated
Lists include: 
* Individual (ind) or Household (hh)
* Continuous (cont) or Categorical (cat)
* All Claritas variables are continous in nature - the aggregation methodology is different and thus requires users to pass specific lists; claritas variables are also classified in Individual or Household)

In [ ]:
ind_cont_list = ['age_trtd']
hh_cont_list = ['ig_ce_family_income_detector']
ind_cat_list = ['age_trtd_bucket']
hh_cat_list = ['ig_ce_family_income_detector_bucket']
ind_cla_list = ['cla_sb002952_p','cla_sb002953_p']
hh_cla_list = ['cla_sb000514_p']

## 3. Calling function to generate summaries

#### 3.1 Select data cut needed (eg: Segment, State etc). This selection must be a column name available in dataframe df.

In [ ]:
user_specificied_cut = 'segmentation_output'

In [ ]:
summary_df = summary(claritas_df,df,user_specificied_cut,
                     ind_cont_list,ind_cat_list,hh_cont_list,hh_cat_list,ind_cla_list,hh_cla_list)

#### 3.2 Writing Summary file to desired location

In [ ]:
summary_df.toPandas().to_csv('Summary_Digital_Labs.csv')